<a href="https://colab.research.google.com/github/iskocyali74/soccercomputervision/blob/main/Bitirme_v2_yeni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOV4 NESNE TANIMA (OBJECT DETECTİON)
Bu notebookta nesne tanıma (object detection) için en hızlı modellerden biri olan yolov4 modeli ile nesne tanıma yapacağız.

# 1.ADIM = DARKNET KLONLAMA VE KURULUMU


In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# ADIM 2: YARDIMCI FONKSİYONLARIN TANIMLANMASI


In [ ]:
# define helper functions
def imShow(path): 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## ADIM 3: FOTOĞRAF TOPLAMA VE ETİKETLEME 
Üzerinde tanıma yapacağımız nesne için bir çok fotoğrafa ihtiyacımız olacak. Ne kadar fazla ve ne kadar farklı tip fotoğrafımız olursa o kadar doğru bir modelimiz olur. 

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## ADIM 4: HAZIRLAMIŞ OLDUĞUMUZ VERİ SETİNİ YÜKLEME
Hazırlamış olduğumuz veri setini google collab'e yüklemeliyiz.

Bunun için drive'ımızda bir yolov4 diye klasör oluşturalım.(bu klasörü daha sonra gerekli dosyaları bir arada toplamak için kullanacağız)

Veri setiniz bir klasör içine alıp zip haline getirerek drive'ımıza yükleyip oradan google collab içine aktarmak bize zaman kazandıracaktır.

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
%cd darknet/
!ls /mydrive/yolov4

/content/darknet
backup	    generate_test.py   obj.data   obj.zip   yolov4-obj.cfg
deneme.png  generate_train.py  obj.names  test.zip


In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

## ADIM 5: EĞİTİM İÇİN GEREKLİ DOSYALARI HAZIRLAYALIM

Bu adımda eğitim için gerekli olan .cfg file, obj.data, obj.names ve train.txt dosyalarını oluşturacağız.

*-Config Dosyası*


In [ ]:
# download cfg to google drive and change its name
%cd darknet/
!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-obj.cfg

Config dosyamızda yapmamız gereken değişiklikler:

(Burada verilen değerler bu değişkenlerin önerilen değerleridir.) 

1.   batch = 64 ve subdivision 16.

3.   max_batches değerini (2000 * eğitilen sınıf sayısı) değerine eşitliyoruz.

4.   steps değerlerini (%80 of max_batches) , (%90 of max_batches) yapıyoruz.

5.   [yolo] başlığı altındaki classes değerlerini eğitim yaptığımız sınıf sayısı ile değiştiriyoruz. 

6.   filters değişkenlerini de (eğitim yapacağımız sınıf sayısı + 5 )*3 değerine eşitliyoruz 



In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

*-obj.names ve obj.data*

yolov3 isimli klasörümüz içine obj.names isimli bir dosya oluşturalım ve dosyayı eğitim yapacağımız nesleriniz isimlerini yazıyoruz.

ÖRNEĞİN:
```
Guitar
Mobile Phone
```

Aynı klasör içinde obj.data isimle bir dosya oluşturarak içine eğitim yapacağımız nesne sayısını, eğitim yaparken kullanacağımız train.txt, text.txt ve obj.names isimli dosyaların adreslerini ve eğitim sonucu bulduğumuz ağırlıkları kaydedeceğimiz dizini yazıyoruz.

ÖRNEĞİN:
```
classes = 2
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive
```

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

*-Train ve Test Dosyaları*


generate_train.py



```
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```

generate_test.py



```
import os

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```


In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

9k.tree     eagle.jpg	 imagenet.labels.list	   person.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  scream.jpg
coco.names  goal.txt	 labels			   voc.names
dog.jpg     horses.jpg	 openimages.names


# ADIM 6: ÖNCEDEN EĞİTİLMİŞ CONVOLUTİONAL KATMANLARIN AĞIRLIKLARINI İNDİRME

Bu adımda önceden eğitilmiş yolov3 için kullanılmış deeplearning katmanları ağırlıklarını indiriyoruz. Bu adımı uygulamak zorunda değiliz ama eğitime bu ağırlıklarla başlamak eğittiğimiz modelin daha doğru çalışmasına ve eğitim süresini kısaltmaya yardımcı olacaktır.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# ADIM 7: KENDİ NESNE TANIYICIMIZI EĞİTELİM

Gerekli tüm dosyalar hazır, eğitime başlayabiliriz.



Eğitimimiz uzun süreceği için google collab bizi serverdan atabilir. Bunun önüne geçmek için aktif olduğumuzu bir şekile bildirmeliyiz.


Bunun için de sayfanın üst tarafına sağ tıklayıp "ögeyi denetle" veya "incele" seçeneğiniz seçip, çıkan pencereden "console"'a tıklayıp açılan komut satırına aşağıdaki kodu ekleyip enter tuşuna basarsak bu kod bizim 10 dakikada bir connect butonuna basarak bizim aktif kalmamızı sağlayacaktır.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# EĞİTİM

Sıradaki komut ile eğitim başlayacaktır.

Eğitimimizin süresi veri setinizdeki fotoğraf sayısı, fotoğrafların kalitesi, eğitim yaptığınız nesne sayısı gibi faktörlere göre değişebilir. Modelimizin doğruluğu için loss değerimiz önemlidir. Loss değerimiz ne kadar düşük olursa modelimiz o kadar doğru çalışır. Modelimizi loss değeri azalmayı durdurana kadar çalıştırıp veri setimize göre mümkün olan en doğru modeli eğitebiliriz. 

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

CUDA status Error: file: ./src/dark_cuda.c : () : line: 38 : build time: Dec 27 2021 - 09:11:57 

 CUDA Error: no CUDA-capable device is detected
Darknet error location: ./src/dark_cuda.c, check_error, line #69
CUDA Error: no CUDA-capable device is detected: Bad file descriptor


Modelimizi eğittikten sonra eğitim sırasında loss değerimizin nasıl değiştiğine dair bir grafik görebiliriz.

In [ ]:
# eğitimimize ait grafiğimiz.
imShow('chart.png')

In [ ]:
# eğitime kaldığımız yerden devam edebiliriz.
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

# ADIM 7: EĞİTTİĞİMİZ MODELİMİZİ KULLANALIM


Eğitimimiz tamamlandı, şimdi istediğimiz fotoğraflar üzerinde tanıma yapabiliriz.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights

# Step 8: Modelimizi Çalıştıralım!!!


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/car4.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/theAIGuysCode/yolov4-deepsort

In [ ]:
%cd yolov4-deepsort/

/content/yolov4-deepsort


In [ ]:
# uncomment below line ONLY if you run into any dependency issues while following tutorial
!pip install -r requirements-gpu.txt

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.3.0rc0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.8.0rc0)
ERROR: No matching distribution found for tensorflow-gpu==2.3.0rc0


In [ ]:
!pip install tensorflow==2.3.0

In [ ]:
!python save_model.py --model yolov4

In [ ]:
!python object_tracker.py --video ./data/video/arsenal.mp4 --output ./outputs/tracker1.avi --model yolov4 --dont_show --info

In [ ]:
import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [ ]:
# convert resulting video from avi to mp4 file format
import os
path_video = os.path.join("outputs","tracker.avi")
%cd outputs/
!ffmpeg -y -loglevel panic -i tracker.avi output.mp4
%cd ..

# output object tracking video
path_output = os.path.join("outputs","output.mp4")
show_video(path_output, width=960)

In [ ]:
!python object_tracker.py --video ./data/video/cars.mp4 --output ./outputs/custom.avi --model yolov4 --dont_show

In [ ]:
path_video = os.path.join("outputs","custom.avi")
%cd outputs/
!ffmpeg -y -loglevel panic -i custom.avi result.mp4
%cd ..

# output object tracking video
path_output = os.path.join("outputs","result.mp4")
show_video(path_output, width=960)